# LightFM Hybrid Model on ML-10m using RePlay Library

In [ ]:
K=10
SEED=1234

The notebook contains an example of LightFM model usage and dataset preprocessing with RePlay, including:
1. Data loading
2. Features preprocessing with pyspark
3. Hyperparameters tuning and building LightFM model based on interaction matrix
4. Model evaluation
5. Hyperparameters tuning and building LightFM model based on interaction matrix and features

# 1) Data loading

We will use MovieLens 10m dataset from rs_datasets package, which contains a list of recommendations datasets.

In [ ]:
from rs_datasets import MovieLens

data = MovieLens("10m")
data.info()

ratings


,user_id,item_id,rating,timestamp
0,1,122,5.0,838985046
1,1,185,5.0,838983525
2,1,231,5.0,838983392



items


,item_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance



tags


,user_id,item_id,tag,timestamp
0,15,4973,excellent!,1215184630
1,20,1747,politics,1188263867
2,20,1747,satire,1188263867


### Convert interaction log to RePlay format

In [ ]:
from replay.data_preparator import DataPreparator

log = DataPreparator().transform(
    data=data.ratings,
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "relevance": "rating",
        "timestamp": "timestamp"
    }
)

### Data split

In [ ]:
from replay.splitters import UserSplitter

user_random_splitter = UserSplitter(
    item_test_size=K,
    user_test_size=500,
    drop_cold_items=True,
    drop_cold_users=True,
    shuffle=True,
    seed=SEED
)

In [ ]:
train, test = user_random_splitter.split(log)
train.count(), test.count()

(9995054, 5000)

In [ ]:
train_opt, val_opt = user_random_splitter.split(train)
train_opt.count(), val_opt.count()

(9990054, 5000)

# 2) Features preprocessing with pyspark

### Convert features to RePlay format

In [ ]:
%%time
item_features = DataPreparator().transform(
    data=data.items,
    columns_names={
        "item_id": "item_id"
    }
)

CPU times: user 47.4 ms, sys: 0 ns, total: 47.4 ms
Wall time: 177 ms


In [ ]:
item_features.show(2)

+-------+--------------------+----------------+
|item_id|              genres|           title|
+-------+--------------------+----------------+
|      1|Adventure|Animati...|Toy Story (1995)|
|      2|Adventure|Childre...|  Jumanji (1995)|
+-------+--------------------+----------------+
only showing top 2 rows



#### Year

In [ ]:
from pyspark.sql import functions as sf
from pyspark.sql.types import IntegerType

In [ ]:
year = item_features.withColumn('year', sf.substring(sf.col('title'), -5, 4).astype(IntegerType())).select('item_id', 'year')
year.show(2)

+-------+----+
|item_id|year|
+-------+----+
|      1|1995|
|      2|1995|
+-------+----+
only showing top 2 rows



#### Genres

In [ ]:
from replay.session_handler import State
from pyspark.sql.functions import split

genres = (
    State().session.createDataFrame(data.items[["item_id", "genres"]])
    .select(
        "item_id",
        split("genres", "\|").alias("genres")
    )
)

In [ ]:
genres.show()

+-------+--------------------+
|item_id|              genres|
+-------+--------------------+
|      1|[Adventure, Anima...|
|      2|[Adventure, Child...|
|      3|   [Comedy, Romance]|
|      4|[Comedy, Drama, R...|
|      5|            [Comedy]|
|      6|[Action, Crime, T...|
|      7|   [Comedy, Romance]|
|      8|[Adventure, Child...|
|      9|            [Action]|
|     10|[Action, Adventur...|
|     11|[Comedy, Drama, R...|
|     12|    [Comedy, Horror]|
|     13|[Animation, Child...|
|     14|             [Drama]|
|     15|[Action, Adventur...|
|     16|      [Crime, Drama]|
|     17|[Comedy, Drama, R...|
|     18|[Comedy, Drama, T...|
|     19|            [Comedy]|
|     20|[Action, Comedy, ...|
+-------+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import explode

genres_list = (
    genres.select(explode("genres").alias("genre"))
    .distinct().filter('genre <> "(no genres listed)"')
    .toPandas()["genre"].tolist()
)

In [ ]:
genres_list

['Documentary',
 'Fantasy',
 'IMAX',
 'Adventure',
 'War',
 'Animation',
 'Comedy',
 'Thriller',
 'Film-Noir',
 'Crime',
 'Sci-Fi',
 'Musical',
 'Mystery',
 'Drama',
 'Horror',
 'Western',
 'Romance',
 'Children',
 'Action']

In [ ]:
from pyspark.sql.functions import col, lit, array_contains
from pyspark.sql.types import IntegerType

item_features = genres
for genre in genres_list:
    item_features = item_features.withColumn(
        genre,
        array_contains(col("genres"), genre).astype(IntegerType())
    )
item_features = item_features.drop("genres").cache()
item_features.count()

10681

In [ ]:
item_features.show(2)

+-------+-----------+-------+----+---------+---+---------+------+--------+---------+-----+------+-------+-------+-----+------+-------+-------+--------+------+
|item_id|Documentary|Fantasy|IMAX|Adventure|War|Animation|Comedy|Thriller|Film-Noir|Crime|Sci-Fi|Musical|Mystery|Drama|Horror|Western|Romance|Children|Action|
+-------+-----------+-------+----+---------+---+---------+------+--------+---------+-----+------+-------+-------+-----+------+-------+-------+--------+------+
|      1|          0|      1|   0|        1|  0|        1|     1|       0|        0|    0|     0|      0|      0|    0|     0|      0|      0|       1|     0|
|      2|          0|      1|   0|        1|  0|        0|     0|       0|        0|    0|     0|      0|      0|    0|     0|      0|      0|       1|     0|
+-------+-----------+-------+----+---------+---+---------+------+--------+---------+-----+------+-------+-------+-----+------+-------+-------+--------+------+
only showing top 2 rows



In [ ]:
item_features = item_features.join(year, on='item_id', how='inner')
item_features.cache()
item_features.count()

10681

# 3) Hyperparameters tuning and building LightFM model based on interaction matrix

In [ ]:
from replay.models import LightFMWrap

# fixing warp loss as it usually shows better performance
model = LightFMWrap(random_state=SEED, loss='warp')

To see the default model search space, get ``_search_space`` attribute

In [ ]:
model._search_space

{'loss': {'type': 'categorical',
  'args': ['logistic', 'bpr', 'warp', 'warp-kos']},
 'no_components': {'type': 'loguniform_int', 'args': [8, 512]}}

We will fix loss function and optimize a number of components only

In [ ]:
best_params = model.optimize(train=train_opt, test=val_opt, param_grid={'no_components':[5, 128]}, budget=10)

In [ ]:
best_params

{'no_components': 63}

In [ ]:
model = LightFMWrap(random_state=SEED, loss='warp', **best_params)

In [ ]:
%%time
model.fit(train)

24-May-21 13:55:23, replay, DEBUG: Начало обучения LightFMWrap
DEBUG:replay:Начало обучения LightFMWrap
24-May-21 13:55:23, replay, DEBUG: Предварительная стадия обучения (pre-fit)
DEBUG:replay:Предварительная стадия обучения (pre-fit)
24-May-21 13:55:27, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)


CPU times: user 31min 25s, sys: 6.92 s, total: 31min 32s
Wall time: 1min 17s


In [ ]:
%%time
recs = model.predict(
    k=K,
    users=test.select('user_id').distinct(),
    log=train,
    filter_seen_items=True
)

24-May-21 13:56:40, replay, DEBUG: Начало предикта LightFMWrap
DEBUG:replay:Начало предикта LightFMWrap


CPU times: user 6.25 s, sys: 1.39 s, total: 7.64 s
Wall time: 26.7 s


# 4) Model evaluation

In [ ]:
from replay.metrics import HitRate, NDCG, MAP
from replay.experiment import Experiment

metrics = Experiment(test, {NDCG(): K,
                            MAP() : K,
                            HitRate(): [1, K]})


In [ ]:
%%time
metrics.add_result("LightFM_no_features", recs)
metrics.results

CPU times: user 74.8 ms, sys: 20.1 ms, total: 94.9 ms
Wall time: 14.3 s


,HitRate@1,HitRate@10,MAP@10,NDCG@10
LightFM_no_features,0.38,0.82,0.126265,0.24204


# 5) Hyperparameters tuning and building LightFM model based on interaction matrix and features

In [ ]:
from replay.models import LightFMWrap

model_feat = LightFMWrap(random_state=SEED, loss='warp')

In [ ]:
best_params = model_feat.optimize(train=train_opt, test=val_opt, param_grid={'no_components':[5, 256]}, budget=10, item_features=item_features)

In [ ]:
best_params

{'no_components': 79}

#### best_params

In [ ]:
model_feat = LightFMWrap(random_state=SEED, **best_params, loss='warp')

In [ ]:
%%time
model_feat.fit(train, item_features=item_features)

24-May-21 15:55:09, replay, DEBUG: Начало обучения LightFMWrap
DEBUG:replay:Начало обучения LightFMWrap
24-May-21 15:55:09, replay, DEBUG: Предварительная стадия обучения (pre-fit)
DEBUG:replay:Предварительная стадия обучения (pre-fit)
24-May-21 15:55:10, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)


CPU times: user 12h 32min 47s, sys: 4min 36s, total: 12h 37min 24s
Wall time: 16min 47s


In [ ]:
%%time
recs = model_feat.predict(
    k=K,
    users=test.select('user_id').distinct(),
    log=train,
    filter_seen_items=True,
    item_features=item_features
)

24-May-21 16:11:56, replay, DEBUG: Начало предикта LightFMWrap
DEBUG:replay:Начало предикта LightFMWrap


CPU times: user 6.14 s, sys: 1.39 s, total: 7.52 s
Wall time: 24.9 s


In [ ]:
metrics.add_result("LightFM_item_features", recs)
metrics.results

,HitRate@1,HitRate@10,MAP@10,NDCG@10
LightFM_no_features,0.380,0.820,0.126265,0.24204
LightFM_item_features,0.288,0.772,0.088089,0.18762


In our experiment item features did not improve quality, but in general features might be helpful especially for making recommendations for cold users.
